# Teoria de Números Computacional 21/22
---
## Trabalho Prático 1

Grupo:

* Ivo Miguel Gomes Lima (A90214)

* Tiago dos Santos Silva Peixoto Carriço (A91695)

---
## Contextualização

Neste trabalho foi-nos pedida a implementação e explicação de um dos ataques feitos ao $RSA$, o famoso ataque de $Hastad$.

Para o efeito recorremos ao [SageMath](https://www.sagemath.org) e a alguns documentos bibliográficos o [D. Boneh, Twenty years of attacks on the RSA cryptosystem](http://crypto.stanford.edu/~dabo/pubs/papers/RSA-survey.pdf) e [Glenn Durfee, Cryptanalysis of RSA Using Algebraic and Lattice Methods (pag 24)](http://theory.stanford.edu/~gdurf/durfee-thesis-phd.pdf) para conseguirmos exemplificar e adquirir a precessão de como o ataque funciona. 

No final do documento apresentamos alguns exemplos a chaves $RSA$ com expoente 3.

### Criação do RSA 

O algoritmo de criptografia Rivest-Shamir-Adleman ($RSA$) é um algoritmo de criptografia assimétrico amplamente utilizado para transmissão segura de dados. A criptografia assimétrica usa um par de chaves matematicamente ligadas para criptografar e descriptografar dados. O $RSA$ envolve um par de chaves, uma chave pública gerada através de dois números primos grandes $p$ e $q$, tendo estes valores a ordem de $10^{100}$, esta chave pode ser conhecida por todos, já a chave privada deve ser mantida em sigilo. Toda mensagem cifrada usando uma chave pública só pode ser decifrada usando a respectiva chave privada. Após a geração destes valores é calculado um $n$ que é a somente a multiplicação dos valores $p$ e $q$, isto é $ n\ =\ p\ \times\ q$. Em seguida devemos calcular a [Função totiente de Euler](https://pt.wikipedia.org/wiki/Função_totiente_de_Euler) que geralmente é apresentada como $\phi{(n)}$ mas nos nosso caso acabamos por chamar $m$, que terá o valor de $(p-1)\ \times\ (q-1)$. Depois utilizamos o valor de expoente ($e$) que nos foi dado no enunciado de 3 que respeita a regra de ser $1 < e < m$. Por fim, temos de calcular o inverso multiplicativo de $e$ em $\mod m$ que será guardado numa variável $d$.

#### Implementação

In [197]:
def RSA(nbits = 512):
    p = random_prime(2^(nbits//2), lbound=2^(nbits//2-1))
    q = random_prime(2^(nbits//2+1), lbound=2^(nbits//2))
    n = p*q
    m = (p-1)*(q-1)
    e = 3
    while gcd(e, m) != 1:
        p = random_prime(2^(nbits//2), lbound=2^(nbits//2-1))
        q = random_prime(2^(nbits//2+1), lbound=2^(nbits//2))
        n = p*q
        m = (p-1)*(q-1)
    d = power_mod(e, -1, m) # é a chave privada, inverso de e mod m
    return (n, e), d

### Encriptação e Decriptação



#### Implementação

In [ ]:
def RSA_encriptar(mens, ch_pub):
    n, e = ch_pub
    cripto = []
    for ch in mens:
        cripto.append( power_mod(ord(ch), e, n) )
    #cripto = power_mod(mens, e, n)
    return cripto

def RSA_desencriptar(cripto, ch_pub, ch_priv):
    n, _ = ch_pub
    decif = []
    for ch in cripto:
        decif.append( chr(power_mod(ch, ch_priv, n)) )
    #decif = power_mod(cripto, ch_priv, n)
    return "".join(decif)

In [238]:
plaintext = '''Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.
Sed ut perspiciatis unde omnis iste natus error sit voluptatem accusantium doloremque laudantium, totam rem aperiam, eaque ipsa quae ab illo inventore veritatis et quasi architecto beatae vitae dicta sunt explicabo. Nemo enim ipsam voluptatem quia voluptas sit aspernatur aut odit aut fugit, sed quia consequuntur magni dolores eos qui ratione voluptatem sequi nesciunt. Neque porro quisquam est, qui dolorem ipsum quia dolor sit amet, consectetur, adipisci velit, sed quia non numquam eius modi tempora incidunt ut labore et dolore magnam aliquam quaerat voluptatem. Ut enim ad minima veniam, quis nostrum exercitationem ullam corporis suscipit laboriosam, nisi ut aliquid ex ea commodi consequatur? Quis autem vel eum iure reprehenderit qui in ea voluptate velit esse quam nihil molestiae consequatur, vel illum qui dolorem eum fugiat quo voluptas nulla pariatur?'''

In [199]:
chaves_publicas = []
chaves_privadas = []
cifras = []

for i in range(3):
    publica, privada = RSA()
    
    chaves_publicas.append(publica)
    chaves_privadas.append(privada)
    cifras.append(RSA_encriptar(plaintext, publica))

In [200]:
#for i in range(3):
#    publica = chaves_publicas[i]
#    privada = chaves_privadas[i]
#    cifra = cifras[i]
#    
#    print(RSA_desencriptar(cifra, publica, privada))

## Ataque de $Hastad$

In [201]:
def hastad(cifras, chaves_publicas):
    res = []
    for i in range(len(cifras[0])):
        x = crt([x[i] for x in cifras], [x[0] for x in chaves_publicas]) # Teorema Chinês dos Restos
        res.append(x.nth_root(3)) # Raiz de grau 3 de x
    return "".join(map(chr, res))

In [202]:
#x = crt(cifras, [x[0] for x in chaves_publicas])
#x.nth_root(3)

In [203]:
hastad(cifras, chaves_publicas)

'wicked'

### Alternativa

Limitada

In [239]:
def RSA_encriptar(mens, ch_pub):
    n, e = ch_pub
    plain = ""
    for ch in mens:
        x = ord(ch)
        plain += format(x, '03')
    cripto = power_mod(int(plain), e, n)
    return cripto

In [240]:
chaves_publicas = []
chaves_privadas = []
cifras = []

for i in range(3):
    publica, privada = RSA()
    
    chaves_publicas.append(publica)
    chaves_privadas.append(privada)
    cifras.append(RSA_encriptar(plaintext, publica))

In [241]:
def convertToString(decif):
    decifString = str(decif)
    
    plain = []
    i = len(decifString)
    
    while i > 0:
        if i >= 3:
            plain.append(chr(int(decifString[i-3:i])))
        else:
            plain.append(chr(int(decifString[0:i])))
        i -= 3
    return "".join(plain[::-1])

In [242]:
def RSA_desencriptar(cripto, ch_priv, ch_pub):
    n, e = ch_pub
    decif = power_mod(cripto, ch_priv, n)
    
    return convertToString(decif)

In [243]:
def hastad(cifras, ch_pub):
    x = crt(cifras, [x[0] for x in ch_pub])
    return convertToString(x.nth_root(3))

In [ ]:
hastad(cifras, chaves_publicas)

## Exemplos

### Exemplo 1

### Exemplo 2

### Exemplo 3